In [1]:
import pandas as pd
import numpy as np 

In [3]:
!pip install openpyxl

Здесь пишете путь, где сохранен excel файл с описанием данных (скачать его можно здесь: https://disk.yandex.ru/i/RTgCJgZOtPo2aA)

In [5]:
df = pd.read_excel(r'C:\Users\12345654321\Desktop\Учеба ВИШ\НИР\База данных МРТ гипофиза.xlsx')

In [7]:
df['Файл с разметкой Ax T1+C'].isna().value_counts()

Файл с разметкой Ax T1+C
False    300
Name: count, dtype: int64

В data_dir пишете название папки, куда будут сохранятся данные и запускайте скрипт. Загружаться будет приблизительно час.

In [9]:
import os
import pandas as pd
import requests
import zipfile
import shutil
from urllib.parse import urlencode
#Assuming your dataframe is df


# Создание папки если она не существует
data_dir = 'data_pituitary_18_01_25'
os.makedirs(data_dir, exist_ok=True)

# Обход DataFrame
for index, row in df.iterrows():
    patient_id = row['ID пациента'] 
    patient_dir = f'{data_dir}/ID_{patient_id}' # путь к директории пациента

    # Создание папки для пациента, если она не существует
    os.makedirs(patient_dir, exist_ok=True)


    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = row['Местоположение файлов']  # Сюда вписываете вашу ссылку

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    # Скачивание и разархивирование файла Если это прямые ссылки на zip файлы
    # response = requests.get(final_url, stream=True)
    response = requests.get(final_url)
    download_url = response.json()['href']
    zip_file_path = os.path.join(patient_dir, 'temp.zip') # Путь к временному zip файлу
    download_response = requests.get(download_url)

    # Сохранение zip файла
    with open(zip_file_path, 'wb') as fd:
        fd.write(download_response.content)

    # Разархивация файла
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(patient_dir)

    # Удаление временного zip файла
    os.remove(zip_file_path)
    
    # # Создание файла с метками класса
    with open(os.path.join(patient_dir, 'labels.txt'), 'w', encoding='utf-8') as f:
        f.write(f'Категория Knosp слева (S): {row["Категория Knosp слева (S)"]}\n')
        f.write(f'Категория Knosp справа (D): {row["Категория Knosp справа (D)"]}\n')